In [1]:
import pandas as pd
import numpy as np

## Load training dataset

In [20]:
dataset_path = './dataset_train.csv'
dataset_train = pd.read_csv(dataset_path)
# dataset = pd.read_csv("Housing.csv")

In [21]:
dataset_path = './dataset_val.csv'
dataset_val = pd.read_csv(dataset_path)

In [29]:
dataset_train = dataset_train[dataset_train["status"] == "sold"]
dataset_val = dataset_val[dataset_val["status"] == "sold"]

In [30]:
# dataset_train
dataset

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [31]:
input_columns = ['bed', 'bath', 'house_size']
# input_columns = ['bedrooms', 'bathrooms', 'area']
output_columns = ['price']

all_columns = input_columns + output_columns

In [32]:
from sklearn.model_selection import train_test_split
TEST_PERC = 0.2
VAL_PERC = 0.1

# dataset_train, dataset_test = train_test_split(dataset, test_size=TEST_PERC, random_state=0)
# dataset_test, dataset_val = train_test_split(dataset_test, test_size=VAL_PERC / (VAL_PERC + TEST_PERC), random_state=0)

In [33]:
dataset_val

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,80325.0,sold,309900.0,3.0,2.0,0.24,1620061.0,West Springfield,Massachusetts,1089.0,1827.0,2021-10-28
1,81557.0,sold,189000.0,3.0,2.0,0.36,276778.0,East Stroudsburg,Pennsylvania,18301.0,1340.0,2021-11-30
2,78818.0,sold,375000.0,3.0,2.0,0.21,1764173.0,Saint Petersburg,Florida,33704.0,1192.0,2022-02-18
3,30807.0,sold,85000.0,2.0,1.0,0.33,563900.0,Florissant,Missouri,63031.0,1167.0,2021-11-05
7,78797.0,sold,164900.0,2.0,1.0,0.23,1375203.0,Springhill,Florida,34608.0,848.0,2022-01-14
...,...,...,...,...,...,...,...,...,...,...,...,...
43643,83598.0,sold,255000.0,3.0,3.0,0.10,729596.0,Norwalk,Iowa,50211.0,1667.0,2022-04-21
43646,73860.0,sold,214500.0,3.0,2.0,NaN,1610079.0,Greenbelt,Maryland,20770.0,1160.0,2022-03-11
43647,58476.0,sold,88900.0,3.0,2.0,0.23,1330248.0,Fairfield,Alabama,35064.0,1548.0,2022-02-01
43648,30788.0,sold,849888.0,5.0,4.0,0.30,42253.0,Fairfax,Virginia,22032.0,2987.0,2022-03-21


In [34]:
cleaned_train = dataset_train[all_columns].dropna()
training_data = np.array(cleaned_train, dtype=np.float32)
x = cleaned_train[input_columns]
x = np.array(x, dtype=np.float32)
y = cleaned_train[output_columns]
y = np.array(y, dtype=np.float32)

cleaned_val = dataset_train[all_columns].dropna()
x_val = cleaned_val[input_columns]
x_val = np.array(x_val, dtype=np.float32)
y_val = cleaned_val[output_columns]
y_val = np.array(y_val, dtype=np.float32)

## Creating model

In [35]:
import keras
from keras import layers

normalization_layer = layers.Normalization()
normalization_layer.adapt(x)

model = keras.Sequential( 
  [ 
    normalization_layer,
    layers.Input(len(input_columns)),
    layers.Dense(128, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(1),
  ] 
) 

In [36]:
# model.compile(
#   optimizer=keras.optimizers.Adam(learning_rate=1e-2),
#   loss=keras.losses.MeanSquaredError(),
#   metrics=keras.metrics.RootMeanSquaredError(),
# )

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])

In [38]:
# h = model.fit(x, y, epochs=500, batch_size=32)

early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

history = model.fit(
  x, y,
  validation_data=(x_val, y_val),
  epochs=1,
  batch_size=32,
  # callbacks=[early_stopping],
)

11996/11996 [==============================] - 203s 17ms/step - loss: 233173073920.0000 - mae: 171026.5781 - val_loss: 231493369856.0000 - val_mae: 171319.6875


In [11]:
# import pickle
# with open('model.pkl', 'wb') as file:
#   pickle.dump(model, file)

In [12]:
model.save('model.keras')